# 📈 Sales Forecasting - Time Series Analysis

**Project**: Sales Forecasting & Business Intelligence Dashboard  
**Author**: [Your Name]  
**Date**: September 2024

---

## 🎯 Notebook Overview

Welcome to my comprehensive sales forecasting analysis! In this notebook, I'll build and compare multiple forecasting models to predict future sales trends. 

### What we'll accomplish:
- 📊 **Exploratory Data Analysis** - Understanding sales patterns and trends
- 🔍 **Time Series Decomposition** - Identifying trend, seasonality, and noise
- 🤖 **Multiple Forecasting Models** - ARIMA, Prophet, Random Forest, Linear Regression
- 📈 **Model Evaluation** - Comparing accuracy and business applicability
- 💡 **Business Insights** - Actionable recommendations for stakeholders

This analysis will demonstrate my ability to extract business value from time series data and build production-ready forecasting systems!

In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Time series and forecasting
from datetime import datetime, timedelta
import warnings
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Prophet for advanced time series forecasting
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except ImportError:
    print("⚠️ Prophet not available. Install with: pip install prophet")
    PROPHET_AVAILABLE = False

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("📈 Sales Forecasting Analysis - Ready to Go!")
print(f"🕐 Analysis started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔮 Prophet available: {PROPHET_AVAILABLE}")

## 📊 Data Generation & Loading

For this demonstration, I'll create realistic sales data that mimics real-world patterns including trends, seasonality, and business cycles.

In [ ]:
# Generate realistic sales data
def generate_sales_data():
    """Generate realistic sales data with trends, seasonality, and noise"""
    
    print("🔄 Generating realistic sales dataset...")
    
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Create date range (3 years of daily data)
    start_date = datetime(2021, 1, 1)
    end_date = datetime(2024, 9, 30)
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    print(f"📅 Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    print(f"📊 Total days: {len(dates)}")
    
    # Base sales level
    base_sales = 3000
    
    # Long-term trend (growth over time)
    trend = np.linspace(0, 500, len(dates))
    
    # Annual seasonality (peak in Q4, low in Q1)
    annual_seasonality = 400 * np.sin(2 * np.pi * np.arange(len(dates)) / 365.25 + np.pi/2)
    
    # Weekly seasonality (higher on weekends)
    weekly_seasonality = 200 * np.sin(2 * np.pi * np.arange(len(dates)) / 7 + np.pi/2)
    
    # Holiday effects (Black Friday, Christmas, etc.)
    holiday_boost = np.zeros(len(dates))
    
    for i, date in enumerate(dates):
        # Black Friday (4th Thursday of November)
        if date.month == 11 and date.weekday() == 4:  # Friday
            thursday = date - timedelta(days=1)
            if 22 <= thursday.day <= 28:  # 4th Thursday
                holiday_boost[i] = 2000
        
        # Christmas season (December)
        elif date.month == 12:
            holiday_boost[i] = 800 * (1 + np.sin(np.pi * date.day / 31))
        
        # Back to school (August)
        elif date.month == 8:
            holiday_boost[i] = 300
        
        # Summer boost (June-July)
        elif date.month in [6, 7]:
            holiday_boost[i] = 200
    
    # Random noise (business volatility)
    noise = np.random.normal(0, 300, len(dates))
    
    # Combine all components
    sales = base_sales + trend + annual_seasonality + weekly_seasonality + holiday_boost + noise
    
    # Ensure no negative sales
    sales = np.maximum(sales, 100)
    
    # Create DataFrame
    sales_data = pd.DataFrame({
        'date': dates,
        'sales': sales.round(2),
        'year': dates.year,
        'month': dates.month,
        'day': dates.day,
        'weekday': dates.weekday,
        'is_weekend': (dates.weekday >= 5).astype(int),
        'is_holiday_season': ((dates.month == 12) | (dates.month == 11)).astype(int)
    })
    
    # Add additional business features
    sales_data['sales_lag1'] = sales_data['sales'].shift(1)
    sales_data['sales_lag7'] = sales_data['sales'].shift(7)
    sales_data['sales_ma7'] = sales_data['sales'].rolling(window=7).mean()
    sales_data['sales_ma30'] = sales_data['sales'].rolling(window=30).mean()
    
    print(f"💰 Average daily sales: ${sales_data['sales'].mean():,.0f}")
    print(f"📊 Sales range: ${sales_data['sales'].min():,.0f} - ${sales_data['sales'].max():,.0f}")
    print(f"📈 Total revenue: ${sales_data['sales'].sum():,.0f}")
    
    return sales_data

# Generate the dataset
df = generate_sales_data()

# Display basic info
print("\n📋 Dataset Overview:")
print(f"   Shape: {df.shape}")
print(f"   Columns: {list(df.columns)}")
print(f"   Date range: {df['date'].min()} to {df['date'].max()}")
print(f"   Missing values: {df.isnull().sum().sum()}")

# Show sample data
print("\n📊 Sample Data:")
display(df.head())

## 📊 Exploratory Data Analysis

Let's explore our sales data to understand patterns, trends, and seasonality that will inform our forecasting models.

In [ ]:
# Set date as index for time series analysis
df_ts = df.set_index('date').copy()

# Basic time series visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('📈 Sales Data Exploratory Analysis', fontsize=16, fontweight='bold')

# 1. Overall time series
axes[0,0].plot(df_ts.index, df_ts['sales'], color='navy', alpha=0.7, linewidth=1)
axes[0,0].set_title('Daily Sales Over Time')
axes[0,0].set_ylabel('Sales ($)')
axes[0,0].grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(range(len(df_ts)), df_ts['sales'], 1)
p = np.poly1d(z)
axes[0,0].plot(df_ts.index, p(range(len(df_ts))), "r--", alpha=0.8, linewidth=2, label='Trend')
axes[0,0].legend()

# 2. Sales distribution
axes[0,1].hist(df_ts['sales'], bins=50, color='skyblue', alpha=0.7, edgecolor='black')
axes[0,1].axvline(df_ts['sales'].mean(), color='red', linestyle='--', 
                  label=f'Mean: ${df_ts["sales"].mean():.0f}')
axes[0,1].axvline(df_ts['sales'].median(), color='green', linestyle='--',
                  label=f'Median: ${df_ts["sales"].median():.0f}')
axes[0,1].set_title('Sales Distribution')
axes[0,1].set_xlabel('Sales ($)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Monthly aggregation
monthly_sales = df_ts.groupby([df_ts.index.year, df_ts.index.month])['sales'].sum()
monthly_dates = pd.to_datetime([f'{year}-{month:02d}-01' for year, month in monthly_sales.index])

axes[1,0].plot(monthly_dates, monthly_sales.values, marker='o', linewidth=2, markersize=4, color='green')
axes[1,0].set_title('Monthly Sales Totals')
axes[1,0].set_ylabel('Monthly Sales ($)')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(True, alpha=0.3)

# 4. Weekly seasonality
weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
weekday_sales = df_ts.groupby('weekday')['sales'].mean()

bars = axes[1,1].bar(range(7), weekday_sales.values, color='orange', alpha=0.7, edgecolor='black')
axes[1,1].set_xticks(range(7))
axes[1,1].set_xticklabels(weekday_names)
axes[1,1].set_title('Average Sales by Day of Week')
axes[1,1].set_ylabel('Average Sales ($)')
axes[1,1].grid(True, alpha=0.3)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + 20,
                   f'${height:.0f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Print key insights
print("🔍 Key Insights from EDA:")
print(f"   💰 Average daily sales: ${df_ts['sales'].mean():,.0f}")
print(f"   📊 Sales volatility (std): ${df_ts['sales'].std():,.0f}")
print(f"   📈 Growth trend: {(df_ts['sales'].iloc[-365:].mean() - df_ts['sales'].iloc[:365].mean()):.0f} $/year")
print(f"   📅 Strongest day: {weekday_names[weekday_sales.idxmax()]} (${weekday_sales.max():.0f})")
print(f"   📅 Weakest day: {weekday_names[weekday_sales.idxmin()]} (${weekday_sales.min():.0f})")
print(f"   🎯 Weekend boost: {((weekday_sales[5:].mean() - weekday_sales[:5].mean()) / weekday_sales[:5].mean() * 100):.1f}%")

## 🔍 Time Series Decomposition

Let's decompose our time series into its fundamental components: trend, seasonality, and residuals. This analysis will help us understand the underlying patterns.

In [ ]:
# Perform seasonal decomposition
print("🔄 Performing seasonal decomposition...")

# Use additive decomposition with weekly seasonality (period=7)
decomposition = seasonal_decompose(df_ts['sales'], model='additive', period=7)

# Create decomposition plot
fig, axes = plt.subplots(4, 1, figsize=(16, 12))
fig.suptitle('📊 Time Series Decomposition Analysis', fontsize=16, fontweight='bold')

# Original time series
decomposition.observed.plot(ax=axes[0], color='navy', title='Original Sales Data')
axes[0].set_ylabel('Sales ($)')
axes[0].grid(True, alpha=0.3)

# Trend component
decomposition.trend.plot(ax=axes[1], color='red', title='Trend Component')
axes[1].set_ylabel('Trend ($)')
axes[1].grid(True, alpha=0.3)

# Seasonal component
decomposition.seasonal.plot(ax=axes[2], color='green', title='Seasonal Component (Weekly)')
axes[2].set_ylabel('Seasonal ($)')
axes[2].grid(True, alpha=0.3)

# Residual component
decomposition.resid.plot(ax=axes[3], color='orange', title='Residual Component')
axes[3].set_ylabel('Residual ($)')
axes[3].set_xlabel('Date')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analyze decomposition components
print("\n📊 Decomposition Analysis:")
print(f"   📈 Trend strength: {1 - (decomposition.resid.var() / decomposition.observed.var()):.3f}")
print(f"   🔄 Seasonal strength: {1 - (decomposition.resid.var() / (decomposition.observed - decomposition.trend).var()):.3f}")
print(f"   📊 Trend range: ${decomposition.trend.min():.0f} - ${decomposition.trend.max():.0f}")
print(f"   🔄 Seasonal range: ${decomposition.seasonal.min():.0f} - ${decomposition.seasonal.max():.0f}")
print(f"   📊 Residual std: ${decomposition.resid.std():.0f}")

# Monthly seasonality analysis
print("\n📅 Monthly Seasonal Patterns:")
monthly_avg = df_ts.groupby(df_ts.index.month)['sales'].mean()
overall_avg = df_ts['sales'].mean()

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for month, avg_sales in monthly_avg.items():
    deviation = ((avg_sales - overall_avg) / overall_avg) * 100
    print(f"   {month_names[month-1]}: ${avg_sales:.0f} ({deviation:+.1f}% vs average)")

## 🧪 Stationarity Testing

Before building ARIMA models, we need to test if our time series is stationary. Non-stationary data needs to be transformed.

In [ ]:
# Test for stationarity using Augmented Dickey-Fuller test
def test_stationarity(timeseries, title):
    """Perform ADF test and visualize stationarity"""
    
    print(f"\n📊 Stationarity Test: {title}")
    print("=" * 40)
    
    # Perform ADF test
    adf_result = adfuller(timeseries.dropna())
    
    print(f"ADF Statistic: {adf_result[0]:.6f}")
    print(f"p-value: {adf_result[1]:.6f}")
    print("Critical Values:")
    for key, value in adf_result[4].items():
        print(f"\t{key}: {value:.3f}")
    
    # Interpret results
    if adf_result[1] <= 0.05:
        print("✅ Series is STATIONARY (reject null hypothesis)")
        stationary = True
    else:
        print("❌ Series is NON-STATIONARY (fail to reject null hypothesis)")
        stationary = False
    
    return stationary, adf_result

# Test original series
is_stationary, adf_original = test_stationarity(df_ts['sales'], "Original Sales Data")

# If not stationary, try differencing
if not is_stationary:
    print("\n🔄 Applying differencing to achieve stationarity...")
    
    # First difference
    df_ts['sales_diff1'] = df_ts['sales'].diff()
    is_stationary_diff1, adf_diff1 = test_stationarity(df_ts['sales_diff1'], "First Differenced Data")
    
    # Second difference if needed
    if not is_stationary_diff1:
        df_ts['sales_diff2'] = df_ts['sales_diff1'].diff()
        is_stationary_diff2, adf_diff2 = test_stationarity(df_ts['sales_diff2'], "Second Differenced Data")

# Visualize stationarity
fig, axes = plt.subplots(3, 1, figsize=(15, 10))
fig.suptitle('📊 Stationarity Analysis', fontsize=16, fontweight='bold')

# Original series
axes[0].plot(df_ts.index, df_ts['sales'], color='navy', alpha=0.7)
axes[0].set_title(f'Original Series (ADF p-value: {adf_original[1]:.4f})')
axes[0].set_ylabel('Sales ($)')
axes[0].grid(True, alpha=0.3)

# First difference
if 'sales_diff1' in df_ts.columns:
    axes[1].plot(df_ts.index, df_ts['sales_diff1'], color='red', alpha=0.7)
    axes[1].set_title(f'First Difference (ADF p-value: {adf_diff1[1]:.4f})')
    axes[1].set_ylabel('First Diff ($)')
    axes[1].grid(True, alpha=0.3)
    
    # Rolling statistics for first difference
    rolling_mean = df_ts['sales_diff1'].rolling(window=30).mean()
    rolling_std = df_ts['sales_diff1'].rolling(window=30).std()
    
    axes[2].plot(df_ts.index, rolling_mean, color='green', label='30-day Rolling Mean')
    axes[2].plot(df_ts.index, rolling_std, color='orange', label='30-day Rolling Std')
    axes[2].set_title('Rolling Statistics (First Difference)')
    axes[2].set_xlabel('Date')
    axes[2].set_ylabel('Value')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Determine optimal differencing order
if is_stationary:
    d_order = 0
    working_series = df_ts['sales']
    print("\n✅ Using original series (d=0)")
elif 'sales_diff1' in df_ts.columns and is_stationary_diff1:
    d_order = 1
    working_series = df_ts['sales_diff1']
    print("\n✅ Using first difference (d=1)")
elif 'sales_diff2' in df_ts.columns and is_stationary_diff2:
    d_order = 2
    working_series = df_ts['sales_diff2']
    print("\n✅ Using second difference (d=2)")
else:
    d_order = 1  # Default fallback
    working_series = df_ts['sales_diff1'] if 'sales_diff1' in df_ts.columns else df_ts['sales']
    print("\n⚠️ Using default differencing (d=1)")

print(f"📊 Optimal differencing order for ARIMA: d={d_order}")

## 🤖 Model Building & Training

Now let's build and train multiple forecasting models to find the best approach for our sales data.

In [ ]:
# Prepare data for modeling
print("🔄 Preparing data for modeling...")

# Split data into train/test sets (80/20 split)
split_date = df_ts.index[int(len(df_ts) * 0.8)]
train_data = df_ts[df_ts.index <= split_date].copy()
test_data = df_ts[df_ts.index > split_date].copy()

print(f"📊 Training data: {len(train_data)} days ({train_data.index.min()} to {train_data.index.max()})")
print(f"📊 Testing data: {len(test_data)} days ({test_data.index.min()} to {test_data.index.max()})")

# Initialize model results dictionary
model_results = {}

# Define evaluation metrics
def calculate_metrics(y_true, y_pred, model_name):
    """Calculate comprehensive evaluation metrics"""
    
    # Handle any NaN values
    valid_mask = ~(np.isnan(y_true) | np.isnan(y_pred))
    y_true_clean = y_true[valid_mask]
    y_pred_clean = y_pred[valid_mask]
    
    if len(y_true_clean) == 0:
        return {}
    
    # Calculate metrics
    mae = mean_absolute_error(y_true_clean, y_pred_clean)
    rmse = np.sqrt(mean_squared_error(y_true_clean, y_pred_clean))
    mape = np.mean(np.abs((y_true_clean - y_pred_clean) / y_true_clean)) * 100
    r2 = r2_score(y_true_clean, y_pred_clean)
    
    # Direction accuracy (did we predict the trend correctly?)
    if len(y_true_clean) > 1:
        true_direction = np.diff(y_true_clean) > 0
        pred_direction = np.diff(y_pred_clean) > 0
        direction_accuracy = np.mean(true_direction == pred_direction) * 100
    else:
        direction_accuracy = 0
    
    metrics = {
        'model': model_name,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'r2': r2,
        'direction_accuracy': direction_accuracy
    }
    
    return metrics

print("\n✅ Data preparation completed!")
print(f"   Training set average: ${train_data['sales'].mean():.0f}")
print(f"   Test set average: ${test_data['sales'].mean():.0f}")

In [ ]:
# Model 1: ARIMA
print("🔄 Training ARIMA Model...")

try:
    # Use automated ARIMA parameter selection or set manually
    # For this demo, we'll use simple parameters based on our stationarity analysis
    arima_order = (1, d_order, 1)  # (p, d, q)
    
    print(f"   ARIMA order: {arima_order}")
    
    # Fit ARIMA model
    arima_model = ARIMA(train_data['sales'], order=arima_order)
    arima_fitted = arima_model.fit()
    
    # Make predictions
    arima_forecast = arima_fitted.forecast(steps=len(test_data))
    arima_conf_int = arima_fitted.get_forecast(steps=len(test_data)).conf_int()
    
    # Calculate metrics
    arima_metrics = calculate_metrics(test_data['sales'].values, arima_forecast, 'ARIMA')
    model_results['ARIMA'] = {
        'predictions': arima_forecast,
        'conf_int': arima_conf_int,
        'metrics': arima_metrics,
        'model': arima_fitted
    }
    
    print(f"   ✅ ARIMA MAPE: {arima_metrics['mape']:.2f}%")
    print(f"   ✅ ARIMA R²: {arima_metrics['r2']:.3f}")
    
except Exception as e:
    print(f"   ❌ ARIMA model failed: {e}")
    model_results['ARIMA'] = None

In [ ]:
# Model 2: Prophet (if available)
if PROPHET_AVAILABLE:
    print("🔄 Training Prophet Model...")
    
    try:
        # Prepare data for Prophet (requires specific format)
        prophet_train = train_data.reset_index()[['date', 'sales']].rename(
            columns={'date': 'ds', 'sales': 'y'}
        )
        
        # Create and fit Prophet model
        prophet_model = Prophet(
            daily_seasonality=True,
            weekly_seasonality=True,
            yearly_seasonality=True,
            changepoint_prior_scale=0.05,  # Controls flexibility
            seasonality_prior_scale=10.0   # Controls seasonality strength
        )
        
        prophet_model.fit(prophet_train)
        
        # Create future dataframe for predictions
        prophet_future = prophet_model.make_future_dataframe(
            periods=len(test_data), freq='D'
        )
        
        # Make predictions
        prophet_forecast = prophet_model.predict(prophet_future)
        
        # Extract test predictions
        prophet_test_pred = prophet_forecast.tail(len(test_data))['yhat'].values
        
        # Calculate metrics
        prophet_metrics = calculate_metrics(test_data['sales'].values, prophet_test_pred, 'Prophet')
        model_results['Prophet'] = {
            'predictions': prophet_test_pred,
            'forecast_df': prophet_forecast,
            'metrics': prophet_metrics,
            'model': prophet_model
        }
        
        print(f"   ✅ Prophet MAPE: {prophet_metrics['mape']:.2f}%")
        print(f"   ✅ Prophet R²: {prophet_metrics['r2']:.3f}")
        
    except Exception as e:
        print(f"   ❌ Prophet model failed: {e}")
        model_results['Prophet'] = None
else:
    print("⚠️ Skipping Prophet model (not installed)")

In [ ]:
# Model 3: Random Forest Regression
print("🔄 Training Random Forest Model...")

try:
    # Prepare features for ML model
    feature_columns = ['year', 'month', 'day', 'weekday', 'is_weekend', 
                      'is_holiday_season', 'sales_lag1', 'sales_lag7', 
                      'sales_ma7', 'sales_ma30']
    
    # Prepare training data (remove NaN values)
    X_train = train_data[feature_columns].dropna()
    y_train = train_data.loc[X_train.index, 'sales']
    
    # Prepare test data
    X_test = test_data[feature_columns].dropna()
    y_test = test_data.loc[X_test.index, 'sales']
    
    print(f"   Training samples: {len(X_train)}")
    print(f"   Test samples: {len(X_test)}")
    print(f"   Features: {feature_columns}")
    
    # Train Random Forest
    rf_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
    
    rf_model.fit(X_train, y_train)
    
    # Make predictions
    rf_predictions = rf_model.predict(X_test)
    
    # Calculate metrics
    rf_metrics = calculate_metrics(y_test.values, rf_predictions, 'Random Forest')
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    model_results['Random Forest'] = {
        'predictions': rf_predictions,
        'metrics': rf_metrics,
        'model': rf_model,
        'feature_importance': feature_importance,
        'test_index': X_test.index
    }
    
    print(f"   ✅ Random Forest MAPE: {rf_metrics['mape']:.2f}%")
    print(f"   ✅ Random Forest R²: {rf_metrics['r2']:.3f}")
    print(f"   🔝 Top feature: {feature_importance.iloc[0]['feature']} ({feature_importance.iloc[0]['importance']:.3f})")
    
except Exception as e:
    print(f"   ❌ Random Forest model failed: {e}")
    model_results['Random Forest'] = None

In [ ]:
# Model 4: Linear Regression (Baseline)
print("🔄 Training Linear Regression Model...")

try:
    # Use same features as Random Forest
    X_train = train_data[feature_columns].dropna()
    y_train = train_data.loc[X_train.index, 'sales']
    X_test = test_data[feature_columns].dropna()
    y_test = test_data.loc[X_test.index, 'sales']
    
    # Scale features for linear regression
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train Linear Regression
    lr_model = LinearRegression()
    lr_model.fit(X_train_scaled, y_train)
    
    # Make predictions
    lr_predictions = lr_model.predict(X_test_scaled)
    
    # Calculate metrics
    lr_metrics = calculate_metrics(y_test.values, lr_predictions, 'Linear Regression')
    
    # Feature coefficients
    feature_coefs = pd.DataFrame({
        'feature': feature_columns,
        'coefficient': lr_model.coef_
    }).sort_values('coefficient', key=abs, ascending=False)
    
    model_results['Linear Regression'] = {
        'predictions': lr_predictions,
        'metrics': lr_metrics,
        'model': lr_model,
        'scaler': scaler,
        'feature_coefs': feature_coefs,
        'test_index': X_test.index
    }
    
    print(f"   ✅ Linear Regression MAPE: {lr_metrics['mape']:.2f}%")
    print(f"   ✅ Linear Regression R²: {lr_metrics['r2']:.3f}")
    print(f"   🔝 Strongest coefficient: {feature_coefs.iloc[0]['feature']} ({feature_coefs.iloc[0]['coefficient']:.1f})")
    
except Exception as e:
    print(f"   ❌ Linear Regression model failed: {e}")
    model_results['Linear Regression'] = None

print("\n🎉 Model training completed!")

## 📊 Model Comparison & Evaluation

Let's compare all our models and identify the best performer for our sales forecasting task.

In [ ]:
# Create model comparison table
print("📊 MODEL PERFORMANCE COMPARISON")
print("=" * 60)

comparison_data = []
valid_models = {k: v for k, v in model_results.items() if v is not None}

for model_name, results in valid_models.items():
    metrics = results['metrics']
    comparison_data.append({
        'Model': model_name,
        'MAE': f"${metrics['mae']:,.0f}",
        'RMSE': f"${metrics['rmse']:,.0f}",
        'MAPE': f"{metrics['mape']:.2f}%",
        'R²': f"{metrics['r2']:.3f}",
        'Direction Accuracy': f"{metrics['direction_accuracy']:.1f}%"
    })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# Find best model (lowest MAPE)
best_model_name = min(valid_models.keys(), 
                     key=lambda x: valid_models[x]['metrics']['mape'])
best_mape = valid_models[best_model_name]['metrics']['mape']

print(f"\n🏆 BEST MODEL: {best_model_name} (MAPE: {best_mape:.2f}%)")

# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('🏆 Model Performance Comparison', fontsize=16, fontweight='bold')

# Extract metrics for plotting
model_names = list(valid_models.keys())
mape_scores = [valid_models[m]['metrics']['mape'] for m in model_names]
r2_scores = [valid_models[m]['metrics']['r2'] for m in model_names]
mae_scores = [valid_models[m]['metrics']['mae'] for m in model_names]
direction_acc = [valid_models[m]['metrics']['direction_accuracy'] for m in model_names]

# MAPE comparison (lower is better)
bars1 = axes[0,0].bar(model_names, mape_scores, color='skyblue', alpha=0.7, edgecolor='black')
axes[0,0].set_title('MAPE - Lower is Better')
axes[0,0].set_ylabel('MAPE (%)')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(True, alpha=0.3)

# Add value labels
for bar, value in zip(bars1, mape_scores):
    height = bar.get_height()
    axes[0,0].text(bar.get_x() + bar.get_width()/2., height + 0.1,
                   f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')

# R² comparison (higher is better)
bars2 = axes[0,1].bar(model_names, r2_scores, color='lightgreen', alpha=0.7, edgecolor='black')
axes[0,1].set_title('R² Score - Higher is Better')
axes[0,1].set_ylabel('R² Score')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].grid(True, alpha=0.3)

for bar, value in zip(bars2, r2_scores):
    height = bar.get_height()
    axes[0,1].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                   f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

# MAE comparison
bars3 = axes[1,0].bar(model_names, mae_scores, color='orange', alpha=0.7, edgecolor='black')
axes[1,0].set_title('MAE - Lower is Better')
axes[1,0].set_ylabel('MAE ($)')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(True, alpha=0.3)

for bar, value in zip(bars3, mae_scores):
    height = bar.get_height()
    axes[1,0].text(bar.get_x() + bar.get_width()/2., height + 20,
                   f'${value:.0f}', ha='center', va='bottom', fontweight='bold')

# Direction Accuracy
bars4 = axes[1,1].bar(model_names, direction_acc, color='purple', alpha=0.7, edgecolor='black')
axes[1,1].set_title('Direction Accuracy - Higher is Better')
axes[1,1].set_ylabel('Direction Accuracy (%)')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3)

for bar, value in zip(bars4, direction_acc):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + 1,
                   f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize predictions vs actual values
fig, axes = plt.subplots(len(valid_models), 1, figsize=(16, 4*len(valid_models)))
if len(valid_models) == 1:
    axes = [axes]

fig.suptitle('🔮 Model Predictions vs Actual Sales', fontsize=16, fontweight='bold')

for i, (model_name, results) in enumerate(valid_models.items()):
    predictions = results['predictions']
    
    # Get the appropriate test data indices
    if 'test_index' in results:
        test_dates = results['test_index']
        actual_values = test_data.loc[test_dates, 'sales']
    else:
        test_dates = test_data.index
        actual_values = test_data['sales']
        # Ensure predictions and actual values have same length
        min_len = min(len(predictions), len(actual_values))
        predictions = predictions[:min_len]
        actual_values = actual_values.iloc[:min_len]
        test_dates = test_dates[:min_len]
    
    # Plot actual vs predicted
    axes[i].plot(test_dates, actual_values, label='Actual', color='navy', linewidth=2, alpha=0.8)
    axes[i].plot(test_dates, predictions, label='Predicted', color='red', linewidth=2, alpha=0.8, linestyle='--')
    
    # Add confidence intervals for ARIMA if available
    if model_name == 'ARIMA' and 'conf_int' in results:
        conf_int = results['conf_int']
        axes[i].fill_between(test_dates, 
                           conf_int.iloc[:, 0], 
                           conf_int.iloc[:, 1], 
                           alpha=0.2, color='red', label='95% Confidence')
    
    axes[i].set_title(f'{model_name} - MAPE: {results["metrics"]["mape"]:.2f}%')
    axes[i].set_ylabel('Sales ($)')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Calculate business insights
print("\n💡 BUSINESS INSIGHTS:")
print("=" * 40)

if valid_models:
    avg_actual = test_data['sales'].mean()
    print(f"📊 Average actual daily sales: ${avg_actual:,.0f}")
    
    for model_name, results in valid_models.items():
        if 'test_index' in results:
            avg_predicted = np.mean(results['predictions'])
        else:
            avg_predicted = np.mean(results['predictions'][:len(test_data)])
        
        bias = ((avg_predicted - avg_actual) / avg_actual) * 100
        print(f"   {model_name}: Avg predicted ${avg_predicted:,.0f} (bias: {bias:+.1f}%)")
    
    print(f"\n🎯 Best model ({best_model_name}) insights:")
    best_results = valid_models[best_model_name]
    print(f"   💰 Forecast accuracy: {100 - best_results['metrics']['mape']:.1f}%")
    print(f"   📈 Direction accuracy: {best_results['metrics']['direction_accuracy']:.1f}%")
    print(f"   📊 Explains {best_results['metrics']['r2']:.1%} of sales variance")

## 🔮 Future Forecasting

Now let's use our best model to generate actual forecasts for the next 30 days and create actionable business recommendations.

In [ ]:
# Generate future forecasts using the best model
print(f"🔮 Generating future forecasts using {best_model_name}...")

forecast_days = 30
last_date = df_ts.index.max()
future_dates = pd.date_range(start=last_date + timedelta(days=1), 
                           periods=forecast_days, freq='D')

best_results = valid_models[best_model_name]

if best_model_name == 'ARIMA':
    # ARIMA forecast
    print("   Using ARIMA model for forecasting...")
    arima_model = best_results['model']
    future_forecast = arima_model.forecast(steps=forecast_days)
    future_conf_int = arima_model.get_forecast(steps=forecast_days).conf_int()
    
elif best_model_name == 'Prophet' and PROPHET_AVAILABLE:
    # Prophet forecast
    print("   Using Prophet model for forecasting...")
    prophet_model = best_results['model']
    
    # Create future dataframe
    future_df = pd.DataFrame({'ds': future_dates})
    future_prophet = prophet_model.predict(future_df)
    future_forecast = future_prophet['yhat'].values
    future_conf_int = future_prophet[['yhat_lower', 'yhat_upper']]
    
else:
    # Machine Learning models need features
    print(f"   Using {best_model_name} model for forecasting...")
    
    # Create future features
    future_features = []
    
    for date in future_dates:
        # Get recent sales data for lag features
        recent_sales = df_ts['sales'].tail(30).values
        
        features = {
            'year': date.year,
            'month': date.month,
            'day': date.day,
            'weekday': date.weekday(),
            'is_weekend': 1 if date.weekday() >= 5 else 0,
            'is_holiday_season': 1 if date.month in [11, 12] else 0,
            'sales_lag1': recent_sales[-1],
            'sales_lag7': recent_sales[-7] if len(recent_sales) >= 7 else recent_sales[-1],
            'sales_ma7': np.mean(recent_sales[-7:]) if len(recent_sales) >= 7 else recent_sales[-1],
            'sales_ma30': np.mean(recent_sales) if len(recent_sales) >= 30 else np.mean(recent_sales)
        }
        future_features.append(features)
    
    future_df = pd.DataFrame(future_features)
    
    # Make predictions
    if best_model_name == 'Linear Regression':
        scaler = best_results['scaler']
        future_scaled = scaler.transform(future_df[feature_columns])
        future_forecast = best_results['model'].predict(future_scaled)
    else:  # Random Forest
        future_forecast = best_results['model'].predict(future_df[feature_columns])
    
    # Simple confidence intervals for ML models (±10%)
    future_conf_int = pd.DataFrame({
        'lower': future_forecast * 0.9,
        'upper': future_forecast * 1.1
    })

# Create forecast results DataFrame
forecast_results = pd.DataFrame({
    'date': future_dates,
    'forecast': future_forecast,
    'lower_bound': future_conf_int.iloc[:, 0] if hasattr(future_conf_int, 'iloc') else future_conf_int['lower'],
    'upper_bound': future_conf_int.iloc[:, 1] if hasattr(future_conf_int, 'iloc') else future_conf_int['upper']
})

print(f"\n📊 30-Day Forecast Summary:")
print(f"   📅 Forecast period: {future_dates[0].strftime('%Y-%m-%d')} to {future_dates[-1].strftime('%Y-%m-%d')}")
print(f"   💰 Average daily forecast: ${future_forecast.mean():,.0f}")
print(f"   📊 Forecast range: ${future_forecast.min():,.0f} - ${future_forecast.max():,.0f}")
print(f"   💼 Total forecasted revenue: ${future_forecast.sum():,.0f}")

# Compare to historical performance
historical_avg = df_ts['sales'].tail(30).mean()
forecast_vs_historical = ((future_forecast.mean() - historical_avg) / historical_avg) * 100

print(f"   📈 vs Last 30 days: {forecast_vs_historical:+.1f}% change")

# Display forecast table
print(f"\n📋 Detailed Forecast (First 10 days):")
forecast_display = forecast_results.head(10).copy()
forecast_display['forecast'] = forecast_display['forecast'].round(0).astype(int)
forecast_display['lower_bound'] = forecast_display['lower_bound'].round(0).astype(int)
forecast_display['upper_bound'] = forecast_display['upper_bound'].round(0).astype(int)
forecast_display['date'] = forecast_display['date'].dt.strftime('%Y-%m-%d')

print(forecast_display.to_string(index=False))

In [ ]:
# Visualize the forecast
fig, axes = plt.subplots(2, 1, figsize=(16, 10))
fig.suptitle('🔮 Sales Forecast Visualization', fontsize=16, fontweight='bold')

# Historical + Forecast plot
historical_period = df_ts.tail(90)  # Last 90 days

axes[0].plot(historical_period.index, historical_period['sales'], 
            label='Historical Sales', color='navy', linewidth=2, alpha=0.8)
axes[0].plot(forecast_results['date'], forecast_results['forecast'], 
            label='Forecast', color='red', linewidth=2, alpha=0.8, linestyle='--')

# Add confidence bands
axes[0].fill_between(forecast_results['date'], 
                    forecast_results['lower_bound'], 
                    forecast_results['upper_bound'], 
                    alpha=0.2, color='red', label='Confidence Interval')

# Add vertical line to separate historical from forecast
axes[0].axvline(x=df_ts.index.max(), color='black', linestyle=':', alpha=0.7, 
               label='Forecast Start')

axes[0].set_title('Historical Sales + 30-Day Forecast')
axes[0].set_ylabel('Sales ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Forecast detail plot
axes[1].plot(forecast_results['date'], forecast_results['forecast'], 
            marker='o', linewidth=2, markersize=4, color='red', alpha=0.8)
axes[1].fill_between(forecast_results['date'], 
                    forecast_results['lower_bound'], 
                    forecast_results['upper_bound'], 
                    alpha=0.3, color='red')

axes[1].set_title('30-Day Forecast Detail')
axes[1].set_ylabel('Sales ($)')
axes[1].set_xlabel('Date')
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

# Add weekend highlighting
for date, forecast in zip(forecast_results['date'], forecast_results['forecast']):
    if date.weekday() >= 5:  # Weekend
        axes[1].plot(date, forecast, marker='s', markersize=6, color='orange', alpha=0.7)

plt.tight_layout()
plt.show()

# Weekly forecast summary
forecast_results['week'] = forecast_results['date'].dt.isocalendar().week
weekly_forecast = forecast_results.groupby('week').agg({
    'forecast': 'sum',
    'date': ['min', 'max']
}).round(0)

weekly_forecast.columns = ['weekly_total', 'week_start', 'week_end']
weekly_forecast['weekly_total'] = weekly_forecast['weekly_total'].astype(int)

print(f"\n📊 Weekly Forecast Breakdown:")
for week, row in weekly_forecast.iterrows():
    print(f"   Week {week}: ${row['weekly_total']:,} ({row['week_start'].strftime('%m/%d')} - {row['week_end'].strftime('%m/%d')})")

## 💼 Business Recommendations

Based on our comprehensive analysis and forecasting models, let me provide actionable business recommendations.

In [ ]:
# Generate comprehensive business recommendations
print("💼 BUSINESS RECOMMENDATIONS & INSIGHTS")
print("=" * 60)

# 1. Forecasting Model Recommendations
print("\n🤖 FORECASTING MODEL INSIGHTS:")
print(f"   🏆 Recommended model: {best_model_name}")
print(f"   📊 Forecast accuracy: {100 - best_results['metrics']['mape']:.1f}%")
print(f"   🎯 Model reliability: {best_results['metrics']['r2']:.1%} variance explained")

if best_results['metrics']['mape'] < 10:
    print("   ✅ Excellent forecasting accuracy - suitable for operational planning")
elif best_results['metrics']['mape'] < 15:
    print("   ✅ Good forecasting accuracy - suitable for strategic planning")
else:
    print("   ⚠️ Moderate accuracy - use with caution and frequent updates")

# 2. Inventory Management
print("\n📦 INVENTORY MANAGEMENT:")
forecasted_total = future_forecast.sum()
current_daily_avg = df_ts['sales'].tail(30).mean()

print(f"   📊 Next 30-day demand: ${forecasted_total:,.0f}")
print(f"   📈 vs Current trend: {forecast_vs_historical:+.1f}% change")

if forecast_vs_historical > 10:
    print("   🔼 INCREASE inventory levels - higher demand expected")
    print(f"   💡 Recommendation: Stock up {abs(forecast_vs_historical):.0f}% more than usual")
elif forecast_vs_historical < -10:
    print("   🔽 REDUCE inventory levels - lower demand expected")
    print(f"   💡 Recommendation: Reduce stock by {abs(forecast_vs_historical):.0f}%")
else:
    print("   ➡️ MAINTAIN current inventory levels - stable demand")
    print("   💡 Recommendation: Continue current procurement strategy")

# 3. Seasonal Insights
print("\n🎭 SEASONAL INSIGHTS:")
weekend_boost = (weekday_sales[5:].mean() - weekday_sales[:5].mean()) / weekday_sales[:5].mean() * 100
print(f"   📅 Weekend sales boost: +{weekend_boost:.1f}%")
print(f"   🎯 Peak day: {weekday_names[weekday_sales.idxmax()]} (${weekday_sales.max():,.0f})")
print(f"   📉 Low day: {weekday_names[weekday_sales.idxmin()]} (${weekday_sales.min():,.0f})")

# Holiday season analysis
if any(date.month in [11, 12] for date in future_dates):
    holiday_forecast = [f for f, date in zip(future_forecast, future_dates) if date.month in [11, 12]]
    if holiday_forecast:
        holiday_avg = np.mean(holiday_forecast)
        regular_avg = np.mean([f for f, date in zip(future_forecast, future_dates) if date.month not in [11, 12]])
        holiday_lift = ((holiday_avg - regular_avg) / regular_avg) * 100
        print(f"   🎄 Holiday season lift: +{holiday_lift:.1f}% vs regular days")

# 4. Risk Assessment
print("\n⚠️ RISK ASSESSMENT:")
forecast_volatility = np.std(future_forecast) / np.mean(future_forecast) * 100
historical_volatility = df_ts['sales'].std() / df_ts['sales'].mean() * 100

print(f"   📊 Forecast volatility: {forecast_volatility:.1f}%")
print(f"   📊 Historical volatility: {historical_volatility:.1f}%")

if forecast_volatility > historical_volatility * 1.2:
    print("   🔴 HIGH RISK: Increased volatility expected")
    print("   💡 Recommendation: Implement flexible inventory strategy")
elif forecast_volatility < historical_volatility * 0.8:
    print("   🟢 LOW RISK: Stable period expected")
    print("   💡 Recommendation: Safe to optimize for efficiency")
else:
    print("   🟡 MODERATE RISK: Normal volatility expected")
    print("   💡 Recommendation: Maintain current risk management practices")

# 5. Actionable Next Steps
print("\n🚀 ACTIONABLE NEXT STEPS:")

steps = [
    f"1. 📊 Implement {best_model_name} model for weekly sales forecasting",
    "2. 📦 Adjust inventory levels based on 30-day forecast",
    "3. 👥 Schedule staff according to predicted busy/slow days",
    "4. 📈 Monitor actual vs predicted sales for model accuracy",
    "5. 🔄 Update forecasts weekly with new sales data"
]

for step in steps:
    print(f"   {step}")

# 6. Financial Impact
print("\n💰 FINANCIAL IMPACT:")
monthly_forecast = forecasted_total
current_monthly = current_daily_avg * 30
revenue_impact = monthly_forecast - current_monthly

print(f"   📈 Forecasted monthly revenue: ${monthly_forecast:,.0f}")
print(f"   📊 Current trend monthly: ${current_monthly:,.0f}")
print(f"   💵 Expected change: ${revenue_impact:+,.0f} ({revenue_impact/current_monthly*100:+.1f}%)")

if revenue_impact > 0:
    print(f"   ✅ Positive outlook - prepare for growth")
else:
    print(f"   ⚠️ Revenue decline expected - implement retention strategies")

print("\n" + "=" * 60)
print("📊 Analysis completed successfully!")
print(f"🕐 Generated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🎯 Ready for business decision-making!")

## 💾 Export Results

Let's save our results for stakeholder sharing and further analysis.

In [ ]:
# Export results to files
print("💾 Exporting analysis results...")

try:
    # 1. Export forecast results
    forecast_export = forecast_results.copy()
    forecast_export['model_used'] = best_model_name
    forecast_export['forecast_accuracy'] = f"{100 - best_results['metrics']['mape']:.1f}%"
    forecast_export.to_csv('sales_forecast_30_days.csv', index=False)
    print("   ✅ sales_forecast_30_days.csv")
    
    # 2. Export model comparison
    comparison_export = pd.DataFrame([
        {
            'model': name,
            'mape': results['metrics']['mape'],
            'rmse': results['metrics']['rmse'],
            'mae': results['metrics']['mae'],
            'r2_score': results['metrics']['r2'],
            'direction_accuracy': results['metrics']['direction_accuracy']
        }
        for name, results in valid_models.items()
    ])
    comparison_export.to_csv('model_performance_comparison.csv', index=False)
    print("   ✅ model_performance_comparison.csv")
    
    # 3. Export historical analysis
    historical_analysis = df_ts[['sales', 'year', 'month', 'weekday', 'is_weekend']].copy()
    historical_analysis['sales_ma7'] = historical_analysis['sales'].rolling(7).mean()
    historical_analysis['sales_ma30'] = historical_analysis['sales'].rolling(30).mean()
    historical_analysis.to_csv('historical_sales_analysis.csv')
    print("   ✅ historical_sales_analysis.csv")
    
    # 4. Export business insights summary
    insights_summary = {
        'analysis_date': datetime.now().strftime('%Y-%m-%d'),
        'best_model': best_model_name,
        'model_accuracy': f"{100 - best_results['metrics']['mape']:.1f}%",
        'forecast_period': f"{future_dates[0].strftime('%Y-%m-%d')} to {future_dates[-1].strftime('%Y-%m-%d')}",
        'total_forecasted_revenue': f"${future_forecast.sum():,.0f}",
        'avg_daily_forecast': f"${future_forecast.mean():,.0f}",
        'vs_historical_trend': f"{forecast_vs_historical:+.1f}%",
        'peak_sales_day': weekday_names[weekday_sales.idxmax()],
        'weekend_boost': f"{weekend_boost:+.1f}%",
        'forecast_volatility': f"{forecast_volatility:.1f}%"
    }
    
    insights_df = pd.DataFrame([insights_summary])
    insights_df.to_csv('business_insights_summary.csv', index=False)
    print("   ✅ business_insights_summary.csv")
    
    # 5. Export feature importance (if available)
    if 'Random Forest' in valid_models and valid_models['Random Forest'] is not None:
        rf_results = valid_models['Random Forest']
        if 'feature_importance' in rf_results:
            rf_results['feature_importance'].to_csv('feature_importance_analysis.csv', index=False)
            print("   ✅ feature_importance_analysis.csv")
    
    print(f"\n📁 All results exported successfully!")
    print(f"🎯 Files ready for stakeholder review and business planning")
    
except Exception as e:
    print(f"❌ Export error: {e}")

print(f"\n🎉 Sales Forecasting Analysis Complete!")
print(f"🔮 30-day forecast generated with {100 - best_results['metrics']['mape']:.1f}% accuracy")
print(f"💼 Business recommendations ready for implementation")
print(f"📊 Thank you for following this comprehensive forecasting journey!")